In [2]:
from glob import glob
import json
from tqdm import tqdm
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

### Load json file Containing Embedding & Text

In [3]:
data = []

with open('ada-002-b.cari.com.my.jsonl') as fopen:
    for x in fopen:
        data.append(json.loads(x))

In [4]:
len(data)

130647

In [5]:
data[1]

{'v': [-0.028657302260398865,
  -0.00593295693397522,
  0.0019334083190187812,
  -0.005465996451675892,
  -0.013445903547108173,
  0.012582346796989441,
  0.023015394806861877,
  -0.024959998205304146,
  -0.019138984382152557,
  -0.01522419136017561,
  0.014392618089914322,
  0.02014966495335102,
  -0.002731399144977331,
  -0.0007656073430553079,
  -0.004851911682635546,
  -0.006780522409826517,
  0.022490864619612694,
  -0.004983044229447842,
  0.027070915326476097,
  -0.012614330276846886,
  0.009358400478959084,
  0.0209940318018198,
  0.00941597018390894,
  -0.01540329959243536,
  -0.02704532817006111,
  -0.015236984938383102,
  -0.0010242746211588383,
  -0.028554953634738922,
  0.009096134454011917,
  -0.021032411605119705,
  0.005641906056553125,
  0.005916965194046497,
  -0.008238974027335644,
  -0.014507759362459183,
  -0.010938390158116817,
  -0.00910892803221941,
  -0.013036513701081276,
  -0.010189973749220371,
  0.011450126767158508,
  -0.0014800408389419317,
  0.0174502506

In [6]:
vectors, texts = [], []

for d in data:
    vectors.append(d['v'])
    texts.append(d['cleaned_text'])

In [7]:
data[0]['cleaned_text']

'mental kut..'

In [8]:
texts[0]

'mental kut..'

In [9]:
from scipy.spatial import KDTree
import numpy as np

In [10]:
concat = np.array(vectors)
concat.shape

(130647, 1536)

### Scipy KDTree for Computing Distance & Fast Nearest Neighbor Lookup

In [11]:
%%time

kd_tree = KDTree(concat, leafsize = 40)

CPU times: user 1.94 s, sys: 20.2 ms, total: 1.96 s
Wall time: 1.97 s
CPU times: user 1.94 s, sys: 20.2 ms, total: 1.96 s
Wall time: 1.97 s


In [12]:
concat[0]

array([-0.01444866, -0.00348269,  0.0260554 , ..., -0.00232234,
       -0.0206106 , -0.04140711])

In [13]:
dist_list = []
ind_list = []
indices = []

def compute_neighbors(args):
    idx, x = args
    dist, ind = kd_tree.query(x, k=len(concat))
    return idx, dist, ind

with ThreadPoolExecutor(max_workers=40) as executor:
    futures = list(tqdm(executor.map(compute_neighbors, enumerate(concat)), total=len(concat)))
    
    for future in tqdm(futures, total=len(concat)):
        if future:
            dist_list.append(future[1])
            ind_list.append(future[2])
            indices.append(future[0])

100%|██████████| 130647/130647 [00:00<00:00, 1083310.73it/s]


In [14]:
len(dist_list),len(ind_list),len(indices)

(130647, 130647, 130647)

### Sort Result Based on Original Index

In [15]:
combined_data = list(zip(indices, dist_list, ind_list))

sorted_data = sorted(combined_data, key=lambda x: x[0])

sorted_indices_list, sorted_dist_list, sorted_index_list = zip(*sorted_data)

In [16]:
texts[8]

'Kenepalan apakah ini??'

In [17]:
texts[118261]

'kegilaan apakah ini??'

In [18]:
texts[70164]

'apakah ini???'

In [31]:
[x for x in sorted_index_list[0][sorted_dist_list[0] > 0.85]]

[94948,
 112318,
 43830,
 38005,
 35290,
 112652,
 50353,
 25920,
 73546,
 8243,
 22901,
 11351]

In [35]:
neg_ = [x for x in sorted_index_list[1][sorted_dist_list[1] > 0.85]]

In [40]:
import random 
if len(neg_) >= 10:
    random_indices = random.sample(neg_, 10)
else:
    random_indices = random.sample(neg_, len(neg))

In [41]:
random_indices

[83642, 32255, 80576, 94337, 22901, 112318, 101171, 52993, 37834, 62847]

### Format Result and Save json file

In [ ]:
def construct_json(index):
    result = {
        'positive': [],
        'negative': []
    }
    
    result['positive'] = [x for x in sorted_index_list[index][sorted_dist_list[index] <= 0.45] if x != index]
    neg_indices = [x for x in sorted_index_list[index][sorted_dist_list[index] > 0.8]]
    
    if len(neg_indices) >= 10:
        neg_indices = random.sample(neg_indices, 10)
    
    result['negative'] = neg_indices
    
    return result


result_json = {int(index): construct_json(index) for index in range(len(data))}

In [47]:
len(result_json)

130647

In [48]:
# Convert int64 to regular integers
def convert_int64(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    raise TypeError

json_result = {int(key): value for key, value in result_json.items()}

with open('b.cari.com.my-pair-mining.json', 'w') as json_file:
    json.dump(json_result, json_file, indent=4, default = convert_int64)